In [1]:
import spacy
import numpy as np
import re

nlp = spacy.load('en_core_web_sm')

In [2]:
def nGrams(doc_string_or_list, n, docIsAlreadyTokenized):
    doc = doc_string_or_list
    if not docIsAlreadyTokenized:
        doc = [token.text for token in nlp(doc_string_or_list)]
    
    doc = [word.lower() for word in doc]
    grams = [doc[i:i+n] for i in range(len(doc)-n+1)]
    return grams
    

In [10]:
doc = 'How old are you today or can you tell me somthing about you?'
n = 3
grams = nGrams(doc, n, False)

In [11]:
print(grams)

[['how', 'old', 'are'], ['old', 'are', 'you'], ['are', 'you', 'today'], ['you', 'today', 'or'], ['today', 'or', 'can'], ['or', 'can', 'you'], ['can', 'you', 'tell'], ['you', 'tell', 'me'], ['tell', 'me', 'somthing'], ['me', 'somthing', 'about'], ['somthing', 'about', 'you'], ['about', 'you', '?']]


In [13]:
from collections import defaultdict

In [18]:
def buildModel():
    model = defaultdict(lambda: defaultdict(lambda : 0))
    return model

In [48]:
def updateCount(nGram, model):
    w_1_to_n_minus_1 = tuple(nGram[:-1])
    w_n = nGram[-1]
    model[w_1_to_n_minus_1][w_n] += 1
    return model

In [49]:
def computeProbability(model):
    for w_1_to_n_minus_1 in model:
        totalCount = float(sum(model[w_1_to_n_minus_1].values()))
        for w_n in model[w_1_to_n_minus_1]:
            model[w_1_to_n_minus_1][w_n] /= totalCount
    return model

In [50]:
import dill

In [51]:
def saveModel(model, fileNameDotPkl):
    with open(fileNameDotPkl, 'wb') as f:
        dill.dump(model, f)
        
def loadModel(model, fileNameDotPkl):
    with open(fileNameDotPkl, 'rb') as f:
        model = dill.load(f)
    return model

In [52]:
from sklearn.datasets import fetch_20newsgroups as getData
from nltk.corpus import reuters as corpus

In [53]:
X, y = getData(subset='train', remove = ('headers', 'footers', 'qoutes'), return_X_y = True)

In [54]:
print(X[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [55]:
len(X)

11314

In [56]:
n = 3
model = buildModel()

In [57]:
for doc in X:
    for nGram in nGrams(doc, n, False):
        model = updateCount(nGram, model)

In [40]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HeitorFelixdeOliveir\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [58]:
for sentence in corpus.sents():
    for nGram in nGrams(sentence, n, True):
        model = updateCount(nGram, model)

In [59]:
model = computeProbability(model)

In [60]:
saveModel(model, 'model.pkl')

In [77]:
text = ['after', 'that']
nextWords = list(model[tuple(text[-n+1:])].keys())
probs = list(model[tuple(text[-n+1:])].values())

In [109]:
nextWord = np.random.choice(nextWords, 1, probs)[0]
print(nextWord)

those


In [130]:
def sampleText(model, startingText = ['after', 'that'], maxLength  = 100, nGramSize=3):
    text = startingText
    n = nGramSize
    while not len(text)>maxLength:
        nextWords = list(model[tuple(text[-n+1:])].keys())
        probs = list(model[tuple(text[-n+1:])].values())
        if len(nextWords) > 0:
            nextWord = np.random.choice(nextWords, 1, probs)[0]
            text.append(nextWord)
        else:
            break
        
    sampled = ' '.join(text)
    return sampled


In [132]:
print(sampleText(model, ['after', 'that']))

after that ! that takes effect , for several days in detention at tulkarm 
 prison , omar jaber lucky : he 'd start a possible scenario for them 
 to exclude the 
 centerpiece of america . a free ride in the beginning i wish these quacks would first use 
 irq7 ! . ' thus ten 
  more bodies . " turkey in cyprus . there exists diffs to the manufacturer     
           ( western galilee ) and check out " . to ensure shipments are kept in close coordination with the tract in general until last wednesday 's 

